Web Crawling & Scraping (Instagram)

This script scrapes Instagram posts using Selenium & BeautifulSoup packages.

Ref (2019): https://medium.com/@srujana.rao2/scraping-instagram-with-python-using-selenium-and-beautiful-soup-8b72c186a058

Ref (2021): https://medium.com/analytics-vidhya/web-scraping-instagram-with-selenium-python-b8e77af32ad4

Ref (2021): http://www.easy2digital.com/automation/python-tutorial-for-digital-marketer-12-using-hashtags-to-scrape-top-instagram-posts-and-instagram-users/

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import re

First, we will specify Instagram profile's username
And scrape the profile's posts.

On your computer, you need to:

1. Install Python selenium package
> conda install -c conda-forge selenium
OR
> pip install selenium

Ref: https://medium.com/@praneeth.jm/running-chromedriver-and-selenium-in-python-on-an-aws-ec2-instance-2fb4ad633bb5

2. Install chromedriver

3. Install Google Chrome

In [ ]:
# Open the web browser
# Selenium uses Chrome Drive to open the profile given a username (public user).
# For example -

username='davidbeckham'
browser = webdriver.Chrome()
browser.get('https://www.instagram.com/'+username+'/?hl=en')

#==== Uncomment the below segment if you wish to programmatically log in ====
#==== Otherwise, when a new browser window pops up - click on Login and manually log in from browser ====
#==== NOTE that you must key in your OWN login username/password ====
#==== A safer method would be to store it in a different file and read from it ====
# time.sleep(5)
# username=browser.find_element_by_css_selector("input[name='username']")
# password=browser.find_element_by_css_selector("input[name='password']")
# username.clear()
# password.clear()
# username.send_keys("YOUR_OWN_INSTAGRAM_USERNAME")
# password.send_keys("YOUR_OWN_INSTAGRAM_PASSWORD")
# login = browser.find_element_by_css_selector("button[type='submit']").click()
# time.sleep(5)
# # save your login info?
# time.sleep(5)
# notnow = browser.find_element_by_xpath("//button[contains(text(), 'Not Now')]").click()

In [ ]:
username='davidbeckham'
browser.get('https://www.instagram.com/'+ username+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# In the new Chrome browser window (used by Selenium)
# Log into Instagram (using your own account)
# Otherwise, crawler will not be able to retrieve posts from multiple pages

In [ ]:
# If you want to open a hashtag page
# hashtag='food'
# browser = webdriver.Chrome()
# browser.get('https://www.instagram.com/explore/tags/'+hashtag)
# Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
'''
Parse HTML source page

Open the source page and use beautiful soup to parse it.

Go through the body of HTML script and extract link for each image in that page
and pass it to an empty list 'links[]'.
'''

links = []
source = browser.page_source
data = bs(source, 'html.parser')

body = data.find('body')
script = body.find_all("div",class_="v1Nh3")


for div in script:
     link = div.find('a')
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))
        
print("Number of Instagram images: ", len(links))

In [ ]:
links

In [ ]:
'''
Remember by default selenium opens only first page.

If you want to scroll through further pages and get more images,
divide the scroll Height by a number and run the parse code multiple times.

This adds new links from each page to the list. For example -
'''


Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight/1.5);")
source = browser.page_source
data = bs(source, 'html.parser')
body = data.find('body')
script = body.find('span')
for link in script.findAll('a'):
     if re.match("/p", link.get('href')):
         links.append('https://www.instagram.com' + link.get('href'))


# === IMPORTANT ===
# sleep time is required.
# If you don't use this, Instagram may interrupt the script and doesn't scroll through pages.

time.sleep(5)



Pagelength = browser.execute_script("window.scrollTo(document.body.scrollHeight/1.5, document.body.scrollHeight/3.0);")
source = browser.page_source
data = bs(source, 'html.parser')

body = data.find('body')
script = body.find_all("div",class_="v1Nh3")

for div in script:
     link = div.find('a')
     if re.match("/p", link.get('href')):
        links.append('https://www.instagram.com' + link.get('href'))
        
print("Number of Instagram images: ", len(links))

In [ ]:
links

In [ ]:
'''
Download images from pandas data frame

Use requests library to download images from the ‘display_url’ in pandas ‘result’ data frame 
and store them with respective shortcode as file name.

(Important Note: Remember that you should respect author’s rights when you 
download copyrighted content. Do not use images/videos from Instagram for commercial intent).
'''
import os
import requests
result.index = range(len(result.index))

path_prefix=""
directory = "Instagram_Photos_" + username

if not os.path.exists(directory):
    os.makedirs(path_prefix + directory)

for i in range(len(result)):
    r = requests.get(result['display_url'][i])
    with open(directory + "/" + result['shortcode'][i]+".jpg", 'wb') as f:
                    f.write(r.content)

Now go check out the folder Instagram_Photos_davidbeckham/

You should see JPG files there.

In [ ]:
'''
Can you go to each post (via its URL) and retrieve metadata such as:
- Number of likes
- Comments (text)
'''

In [ ]:
# The below code segment is by TA Thao (IS434 - G2) - thank you!

for i in range(5):
    name = links[i][28:-1]
    browser.get(links[i])
    browser.implicitly_wait(30) 
    
    print("Post at:", links[i])

    source = browser.page_source
    data = bs(source, 'html.parser')
    body = data.find('body')
    caption = body.find("div", class_="MOdxS").text
    print("Caption is:", caption)
    
    comments = browser.find_element_by_class_name("XQXOT").find_elements_by_class_name("Mr508")[:5]
    print("5 comments are:")
    for comment in comments:
        script = comment.find_element_by_class_name("C4VMK")
        poster = script.find_element_by_tag_name("h3").find_element_by_tag_name("span").text
        post = script.find_element_by_class_name("MOdxS ").text
        print('\t', poster, ":", post)

    likes = browser.find_element_by_xpath("//section[@class='EDfFK ygqzn']").find_element_by_tag_name("span").text
    likes = int(re.sub('[a-z,]+',  '', likes))
    print("Number of likes is", likes)
    time.sleep(5)